In [24]:
import os
import libsql
import json
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
from datetime import datetime
from zoneinfo import ZoneInfo

def convert_time(str_data_UnixTime):
    dt_data = datetime.fromtimestamp(float(str_data_UnixTime), ZoneInfo("America/New_York"))
    day_of_year = int(dt_data.timetuple().tm_yday)
    time_of_day = round(float(dt_data.hour + dt_data.minute/60 + dt_data.second/3600), 3)
    return day_of_year, time_of_day

In [26]:
OPEN_WEATHER_MAP_KEY = os.getenv('OPEN_WEATHER_MAP_KEY')

def request_weather_cloud(str_data_UnixTime):
    # Get Pittsburgh Coordinates
    lat = 40.4406
    lon = -79.9959
    
    # Call weather API
    url = "https://api.openweathermap.org/data/3.0/onecall/timemachine"
    params = {
        "lat": lat,
        "lon": lon,
        "dt": int(str_data_UnixTime),
        "appid": OPEN_WEATHER_MAP_KEY,
    }
    response = requests.get(url, params=params)
    data = response.json()
    weather_main = data['data'][0]['weather'][0]['main']
    cloud_cover_percent = data['data'][0]['clouds']
    
    # Convert weather to int
    # Get weather conditions from website
    list_weather_classes = ['Clear', 'Clouds', 'Rain', 'Snow', 'Drizzle',
                   'Thunderstorm', 'Mist', 'Fog', 'Haze']
    dict_weather_classes_index = {label: idx for idx, label in enumerate(list_weather_classes)}
    weather_index = dict_weather_classes_index.get(weather_main, -1)

    # print("Weather:", weather_main)
    # print("Cloud Cover (%):", cloud_cover_percent)
    return weather_index, cloud_cover_percent

### JS data

In [27]:
url_js = os.getenv("CLAUDELIGHT_DB_URL_JS")
auth_token_js = os.getenv("CLAUDELIGHT_RO_JS")
conn_js = libsql.connect("claude_light_js.db", sync_url=url_js, auth_token=auth_token_js)
conn_js.sync()

In [28]:
cursor = conn_js.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:")
for t in tables:
    print(t[0])

Tables in database:
measurements


In [29]:
data = conn_js.execute("""SELECT * FROM measurements;""").fetchall()
print(len(data))
data

3


[(1,
  '{\n\t"version":"15b32d0bbac2b03ca3f7817ea783556955f907af426e1c1cd0ce18d18551dadc",\n\t"func":"measurementJSON[{r_, g_, b_}, tag_:Null] := With[{meta = metadata[\\"measurementJSON\\", \\"tutorial\\"], data = measurement[{r, g, b}]}, ExportString[Join[meta, data], \\"JSON\\", \\"Compact\\" -> 1]]\\n \\nmetadata[versionedFunction_String, tag_:Null] := With[{functionCode = ToString[InputForm[FullDefinition[versionedFunction]]]}, {\\"version\\" -> Hash[functionCode, \\"SHA256\\", \\"HexString\\"], \\"func\\" -> functionCode, \\"t0\\" -> AbsoluteTime[], \\"user\\" -> $Username, \\"hostname\\" -> $MachineName, \\"tag\\" -> tag}]\\n \\nmeasurement[{r_, g_, b_}] := URLExecute[\\"https:\\/\\/claude-light.cheme.cmu.edu\\/api\\", {\\"R\\" -> r, \\"G\\" -> g, \\"B\\" -> b}]",\n\t"t0":3.962965456853578e9,\n\t"user":"jschrier",\n\t"hostname":"jschrier-laptop",\n\t"tag":"tutorial",\n\t"in":[0.1,0.2,0.3],\n\t"out":{"415nm":937,"445nm":9974,"480nm":5835,"515nm":14290,"555nm":3535,"590nm":3573,"6

In [30]:
data = conn_js.execute("""select * from measurements where json_extract(data, '$.tag') IS NOT NULL""").fetchall()

In [31]:
j_data = json.loads(data[1][1])
j_data['t0']

3962967340.607998

In [32]:
j_data

{'version': '15b32d0bbac2b03ca3f7817ea783556955f907af426e1c1cd0ce18d18551dadc',
 'func': 'measurementJSON[{r_, g_, b_}, tag_:Null] := With[{meta = metadata["measurementJSON", "tutorial"], data = measurement[{r, g, b}]}, ExportString[Join[meta, data], "JSON", "Compact" -> 1]]\n \nmetadata[versionedFunction_String, tag_:Null] := With[{functionCode = ToString[InputForm[FullDefinition[versionedFunction]]]}, {"version" -> Hash[functionCode, "SHA256", "HexString"], "func" -> functionCode, "t0" -> AbsoluteTime[], "user" -> $Username, "hostname" -> $MachineName, "tag" -> tag}]\n \nmeasurement[{r_, g_, b_}] := URLExecute["https://claude-light.cheme.cmu.edu/api", {"R" -> r, "G" -> g, "B" -> b}]',
 't0': 3962967340.607998,
 'user': 'jschrier',
 'hostname': 'jschrier-laptop',
 'tag': 'tutorial',
 'in': [0.4, 0.5, 0.6],
 'out': {'415nm': 1733,
  '445nm': 19251,
  '480nm': 12110,
  '515nm': 33755,
  '555nm': 6944,
  '590nm': 8829,
  '630nm': 14631,
  '680nm': 3248,
  'clear': 59952,
  'nir': 5184}}

In [33]:
dt_data = datetime.fromtimestamp(float(j_data['t0']), ZoneInfo("America/New_York"))
dt_data
# Weird time

datetime.datetime(2095, 7, 31, 12, 15, 40, 607998, tzinfo=zoneinfo.ZoneInfo(key='America/New_York'))

In [34]:
conn_js.close()
# df.to_csv('data_processed/full_data_JS.csv', index=False)

### HL data

In [35]:
url_hl = os.getenv("CLAUDELIGHT_DB_URL_HL")
auth_token_HL = os.getenv("CLAUDELIGHT_RO_HL")
conn_hl = libsql.connect("claude_light_hl.db", sync_url=url_hl, auth_token=auth_token_HL)
conn_hl.sync()

In [36]:
cursor = conn_hl.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in database:")
for t in tables:
    print(t[0])

Tables in database:
df_results_bsweep
df_results_gsweep
df_results_rsweep
df_results_repeat
df_results_lhs


In [37]:
cursor.execute("PRAGMA table_info(df_results_gsweep);")
columns = [row[1] for row in cursor.fetchall()]

print("Columns in df_results_gsweep:")
for col in columns:
    print(f"- {col}")

Columns in df_results_gsweep:
- id
- tag
- in
- temperature_2m
- cloudcover
- weathercode
- datetime
- out


In [40]:
data = conn_hl.execute("""SELECT * FROM df_results_gsweep;""").fetchall()
print(len(data))
data[0:2]

20


[(0,
  'g_sweep',
  '"[0.0, 0.0, 0.0]"',
  30.9,
  83.0,
  3,
  '2025-07-29 16:27',
  '"{\'415nm\': 292, \'445nm\': 549, \'480nm\': 682, \'515nm\': 956, \'555nm\': 1811, \'590nm\': 1961, \'630nm\': 2074, \'680nm\': 980, \'clear\': 7399, \'nir\': 1614}"'),
 (1,
  'g_sweep',
  '"[0.0, 0.05263157894736842, 0.0]"',
  30.9,
  83.0,
  3,
  '2025-07-29 16:27',
  '"{\'415nm\': 347, \'445nm\': 570, \'480nm\': 1367, \'515nm\': 4691, \'555nm\': 2401, \'590nm\': 2038, \'630nm\': 2150, \'680nm\': 1055, \'clear\': 10427, \'nir\': 1688}"')]

In [41]:
data[0][2]

'"[0.0, 0.0, 0.0]"'

In [42]:
data[0][6]

'2025-07-29 16:27'

In [43]:
data[0][7]

'"{\'415nm\': 292, \'445nm\': 549, \'480nm\': 682, \'515nm\': 956, \'555nm\': 1811, \'590nm\': 1961, \'630nm\': 2074, \'680nm\': 980, \'clear\': 7399, \'nir\': 1614}"'

In [56]:
# HL_db_table = "bsweep"
# HL_db_table = "gsweep"
# HL_db_table = "rsweep"
# HL_db_table = "repeat"
HL_db_table = "lhs"

In [57]:
import ast

df_HL = pd.DataFrame()
for row in conn_hl.execute(f"""SELECT * FROM df_results_{HL_db_table};""").fetchall():
    rgb_str = row[2]
    time_str = row[6]
    out_str = row[7]
    
    R, G, B = json.loads(json.loads(rgb_str))
    
    out_dict = ast.literal_eval(out_str) 
    selected_outputs = ['445nm', '515nm', '630nm', 'clear']

    inner_str = ast.literal_eval(out_str)
    out_dict = ast.literal_eval(inner_str)
    selected_outputs = ['445nm', '515nm', '630nm', 'clear']
    out_values = [out_dict[k] for k in selected_outputs]

    dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M")
    str_data_UnixTime = dt.timestamp()
    
    day_of_year, time_of_day = convert_time(str_data_UnixTime)
    weather_index, cloud_cover_percent = request_weather_cloud(str_data_UnixTime)
    
    flat_data = {
        'R': R,
        'G': G,
        'B': B,
        'day_of_year': day_of_year,
        'time_of_day': time_of_day,
        'weather_index': weather_index,
        'cloud_cover_percent': cloud_cover_percent,
        'I_445': out_values[0],
        'I_515': out_values[1],
        'I_630': out_values[2],
        'I_clear': out_values[3]
    }
    df_oneRow = pd.DataFrame([flat_data])
    
    df_HL = pd.concat([df_HL, df_oneRow], ignore_index=True)

print(df_HL.shape)
df_HL.head(3)

(50, 11)


,R,G,B,day_of_year,time_of_day,weather_index,cloud_cover_percent,I_445,I_515,I_630,I_clear
0,0.248,0.317,0.640,212,13.467,1,20,21153,23569,11708,61109
1,0.036,0.811,0.710,212,13.483,1,20,23377,55096,6207,65535
2,0.806,0.234,0.296,212,13.483,1,20,10558,18022,27610,58082


In [58]:
df_HL.to_csv(f'data_processed/full_data_HL_{HL_db_table}.csv', index=False)

### All team data

In [59]:
df_results_bsweep = pd.read_csv('data_processed/full_data_HL_bsweep.csv')
df_results_gsweep = pd.read_csv('data_processed/full_data_HL_gsweep.csv')
df_results_rsweep = pd.read_csv('data_processed/full_data_HL_rsweep.csv')
df_results_repeat = pd.read_csv('data_processed/full_data_HL_repeat.csv')
df_results_lhs = pd.read_csv('data_processed/full_data_HL_lhs.csv')

In [60]:
df_BZ = pd.read_csv("data_processed/full_data.csv")

df_team = pd.concat([df_BZ, df_results_bsweep, df_results_gsweep, df_results_rsweep, df_results_repeat, df_results_lhs])
print(df_team.shape)
df_team.head()

(218, 11)


,R,G,B,day_of_year,time_of_day,weather_index,cloud_cover_percent,I_445,I_515,I_630,I_clear
0,0.0,0.0,0.0,212,13.326,1,96,844,1689,2127,13266
1,0.9,0.5,0.6,212,13.911,1,100,19888,35221,30901,65535
2,1.0,0.8,0.2,212,13.911,1,100,7964,54316,34207,65535
3,0.8,1.0,0.6,212,13.911,1,100,20253,65535,28784,65535
4,0.0,0.1,0.9,212,13.911,1,100,28791,9686,3991,53746


In [61]:
df_team.to_csv("data_processed/full_data_team.csv", index=False)